In [68]:
import pandas as pd
import numpy as np
from glob import glob


In [2]:
import os

from zipline.utils.run_algo import load_extensions
from zipline.utils.calendar_utils import register_calendar, get_calendar

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

In [3]:
xdse = get_calendar("XDSE")

In [13]:
dates = xdse.schedule.loc["2012": "2021"].market_close.index


2365

In [10]:
df = pd.read_csv("./dse_bundles/data/daily/BATBC.csv", index_col='date', parse_dates=True)

In [12]:
df.index

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-08', '2012-01-09', '2012-01-10',
               '2012-01-11', '2012-01-12',
               ...
               '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22',
               '2021-12-23', '2021-12-26', '2021-12-27', '2021-12-28',
               '2021-12-29', '2021-12-30'],
              dtype='datetime64[ns]', name='date', length=2365, freq=None)

In [20]:
dates = pd.to_datetime(dates.date)

In [21]:
df.loc[dates, :]

,open,high,low,close,volume
2012-01-01,42.054283,42.054283,41.225232,41.907193,24255.0
2012-01-02,41.472610,41.472610,41.218546,41.405751,25740.0
2012-01-03,41.325520,41.673187,41.318834,41.439180,37125.0
2012-01-04,41.385693,41.719988,41.385693,41.512725,73755.0
2012-01-05,41.619700,41.619700,41.318834,41.419123,31680.0
...,...,...,...,...,...
2021-12-26,639.000000,639.000000,625.100000,627.200000,207369.0
2021-12-27,626.300000,633.700000,622.500000,623.800000,167286.0
2021-12-28,624.000000,640.000000,624.000000,637.200000,88461.0
2021-12-29,640.000000,647.000000,630.800000,634.500000,120326.0


In [22]:
acfl = pd.read_csv("./dse_bundles/data/daily/ACFL.csv", index_col='date', parse_dates=True)

In [37]:
start = pd.to_datetime(acfl.index[0].date()).tz_localize("UTC")
end = pd.to_datetime(acfl.index[-1].date()).tz_localize("UTC")

In [38]:
start

Timestamp('2018-08-06 00:00:00+0000', tz='UTC')

In [39]:
end

Timestamp('2021-12-30 00:00:00+0000', tz='UTC')

In [59]:
dates = pd.to_datetime(xdse.schedule.loc[start:end].index.date)
temp_df = pd.DataFrame(index=dates, columns=acfl.columns)
temp_df.loc[acfl.index, :] = acfl.values
temp_df['volume'] = temp_df['volume'].fillna(0.0)
temp_df = temp_df.fillna(method='ffill')

In [75]:
acfl[acfl.index.isin(dates)]

,open,high,low,close,volume
date,,,,,
2018-08-06,51.759140,69.874839,51.759140,64.095068,7084025.0
2018-08-07,65.475312,73.584244,65.475312,71.600144,1758603.0
2018-08-08,71.513879,73.239183,64.440129,64.612660,4417177.0
2018-08-09,63.836273,65.389047,58.487828,61.507111,2531795.0
2018-08-12,62.887355,62.887355,55.382280,55.468545,2267080.0
...,...,...,...,...,...
2021-12-26,30.000000,30.000000,28.700000,29.400000,98035.0
2021-12-27,29.400000,29.700000,28.100000,28.500000,112194.0
2021-12-28,28.600000,31.100000,28.600000,30.700000,109749.0


In [73]:
temp_df

,open,high,low,close,volume
2018-08-06,51.759140,69.874839,51.759140,64.095068,7084025.0
2018-08-07,65.475312,73.584244,65.475312,71.600144,1758603.0
2018-08-08,71.513879,73.239183,64.440129,64.612660,4417177.0
2018-08-09,63.836273,65.389047,58.487828,61.507111,2531795.0
2018-08-12,62.887355,62.887355,55.382280,55.468545,2267080.0
...,...,...,...,...,...
2021-12-26,30.000000,30.000000,28.700000,29.400000,98035.0
2021-12-27,29.400000,29.700000,28.100000,28.500000,112194.0
2021-12-28,28.600000,31.100000,28.600000,30.700000,109749.0
2021-12-29,31.000000,31.000000,29.800000,30.100000,192060.0


In [77]:
files = glob('./dse_bundles/data_ex/daily/*')
xdse = get_calendar('XDSE')
for each in files:
    f =  each.split("/")[-1]
    df = pd.read_csv(each, index_col='date', parse_dates=True)
    start = pd.to_datetime(df.index[0].date()).tz_localize("UTC")
    end = pd.to_datetime(df.index[-1].date()).tz_localize("UTC")
    dates = pd.to_datetime(xdse.schedule[start:end].index.date)
    temp_df = pd.DataFrame(index=dates, columns=df.columns)
    df = df[df.index.isin(dates)]
    temp_df.loc[df.index, :] = df.values
    temp_df['volume'] = temp_df['volume'].fillna(0.0)
    temp_df = temp_df.fillna(method='ffill')
    temp_df.to_csv(f"./dse_bundles/data/daily/{f}")

    
